In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
%matplotlib inline


from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn import set_config
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, Ridge, Lasso


import optuna
import statsmodels.api as sm
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator

from sklearn.experimental import enable_iterative_imputer  # still experimental 
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import KFold
import category_encoders as ce

plt.rc('font', family='Malgun Gothic') # 폰트 지정
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format='retina' # 그래프 글씨 뚜렷

In [2]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [3]:
X_train_a, X_test_a = X_train.copy(), X_test.copy()
X = pd.concat([X_train_a,X_test_a], ignore_index=True)

#### 대학성적 결측값 각 대학 평균점수로 변경

In [4]:
univ_lst = X['출신대학'].unique().tolist()
univ_score = []

for univ in univ_lst:
    mean_score = X['대학성적'][X['출신대학'] == univ].mean()
    univ_score.append(mean_score)
    
X['대학성적'].fillna(' ', inplace=True)

for univ, score in zip(univ_lst, univ_score):
    X[X['출신대학'] == univ] = X[X['출신대학'] == univ].replace({'대학성적':' '},score)

#### 근무형태  결측값 처리

In [5]:
X.근무형태 = X.근무형태.fillna('기타')

#### 어학시험 결측값 및 이상값 처리

In [6]:
X.어학시험 = X.어학시험.fillna('없음')

X.어학시험 = X.어학시험.replace(' ', '없음')
X.어학시험 = X.어학시험.replace('기타시험', '기타')

#### 직무태그 결측값  처리

In [7]:
X.직무태그 = X.직무태그.fillna('없음')

#### 근무경력 년(y) / 월(m) / 일(d)로 분류하고 feature 생성

In [8]:
for i in range(len(X['근무경력'])):
    if len(X['근무경력'][i]) == 3 or len(X['근무경력'][i]) == 4:
        X['근무경력'][i] = '0년 ' + X['근무경력'][i]

In [9]:
# year, month로 구분하여 열 생성
we = X['근무경력'].str.split(' ',expand=True)
we.columns = ['년','개월']

we_y = we.년.str.split('년', expand=True) 
we_y.columns = ['년','삭제']
we_y.drop('삭제',axis=1,inplace=True)
we_y = we_y.astype(int) 

we_m = we.개월.str.split('개월', expand=True)
we_m.columns = ['개월','삭제']
we_m.drop('삭제',axis=1,inplace=True)
we_m = we_m.astype(int)

X['근무경력_y'] = we_y['년']
X['근무경력_m'] = we_m['개월']
X['근무경력_d'] = we_y['년'] * 365 + we_m['개월'] * 30

X.drop('근무경력', axis=1, inplace=True)

### 대학전공

1) 대학전공 계열별로 전처리
- 인문계열, 사회계열, 교육계열, 자연계열, 공학계열, 의학계열 예체능 계열, 의학계열
- 대학전공 키워드 포함하면 그 키워드로 변환

#### 인문계열

In [10]:
a = X[X['대학전공'].str.contains('국어|국문|한국어')]['대학전공']
X['대학전공'] = X['대학전공'].replace(list(a),'국어국문학과')
a = X[X['대학전공'].str.contains('중어|중문|중국|한문|Chinese|중국어')]['대학전공']
X['대학전공'] = X['대학전공'].replace(list(a),'중문학과')
a = X[X['대학전공'].str.contains('불어|프랑스어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'불어학과')
a = X[X['대학전공'].str.contains('영어|영문|영미|어학연수|English')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'영문학과')
a = X[X['대학전공'].str.contains('일어|일본어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'일본어학과')
a = X[X['대학전공'].str.contains('스페인어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'스페인어학과')
a = X[X['대학전공'].str.contains('독어|독문|독일어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'독어독문학과')
a = X[X['대학전공'].str.contains('러시아|러시아어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'러시아어학과')
a = X[X['대학전공'].str.contains('고고학|국사|사학|역사')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'역사학과')
a = X[X['대학전공'].str.contains('한문|한자')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'한문학과')
a = X[X['대학전공'].str.contains('인문')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'인문학과')
a = X[X['대학전공'].str.contains('철학|윤리')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'철학과')
a = X[X['대학전공'].str.contains('심리')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'심리학과')
a = X[X['대학전공'].str.contains('문예|창작')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'문예창작과')
a = X[X['대학전공'].str.contains('문헌정보|문헌')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'문헌학과')
a = X[X['대학전공'].str.contains('문화|미술사학|민속')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'민속학과')

#### 사회계열

In [11]:
a = X[X['대학전공'].str.contains('관광|호텔')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'호텔관광학과')
a = X[X['대학전공'].str.contains('행정|핼정학|Public Administration')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'행정학과')
a = X[X['대학전공'].str.contains('신문|방송')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'신문방송학과')
a = X[X['대학전공'].str.contains('언론')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'언론학과')
a = X[X['대학전공'].str.contains('미디어|정보미디어|콘텐츠')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'미디어학과')
a = X[X['대학전공'].str.contains('법학|법률')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'법학과')
a = X[X['대학전공'].str.contains('경제|경상')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'경제학과')
a = X[X['대학전공'].str.contains('경영|MANAGEMENT|경역')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'경영학과')
a = X[X['대학전공'].str.contains('통상|무역|물류|Trade')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'무역학과')
a = X[X['대학전공'].str.contains('사회')]['대학전공']
X['대학전공'] = X['대학전공'].replace(list(a),'사회학과')
a = X[X['대학전공'].str.contains('재무')]['대학전공']
X['대학전공'] = X['대학전공'].replace(list(a),'재무학과')
a = X[X['대학전공'].str.contains('회계|세무')]['대학전공']
X['대학전공'] = X['대학전공'].replace(list(a),'회계학과')
a = X[X['대학전공'].str.contains('정치|외교|정외')]['대학전공']
X['대학전공'] = X['대학전공'].replace(list(a),'정치학과')
a = X[X['대학전공'].str.contains('경찰')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'경찰학과')
a = X[X['대학전공'].str.contains('광고|홍보|광보홍보')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'광고홍보학과')
a = X[X['대학전공'].str.contains('국제')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'국제학과')
a = X[X['대학전공'].str.contains('노인|복지')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'노인복지학과')
a = X[X['대학전공'].str.contains('비서')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'비서학과')
a = X[X['대학전공'].str.contains('사회복지')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'사회복지학과')
a = X[X['대학전공'].str.contains('사회')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'사회')
a = X[X['대학전공'].str.contains('지리|지리학|지적')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'지리학과')
a = X[X['대학전공'].str.contains('항공','항공서비스')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'항공학과')

#### 교육계열

In [12]:
a = X[X['대학전공'].str.contains('교육|사범')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'교육학과')

#### 자연계열

In [13]:
a = X[X['대학전공'].str.contains('가정관리|가정')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'가정관리학과')
a = X[X['대학전공'].str.contains('농업|농과대|농생물|농업학|농학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'농업학과')
a = X[X['대학전공'].str.contains('물리|physics')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'물리학과')
a = X[X['대학전공'].str.contains('산림|원예')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'산림학과')
a = X[X['대학전공'].str.contains('생명')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'생명학과')
a = X[X['대학전공'].str.contains('수산')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'수산학과')
a = X[X['대학전공'].str.contains('식품|영양')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'식품영양')
a = X[X['대학전공'].str.contains('조리')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'조리학과')
a = X[X['대학전공'].str.contains('지구|지구과학|지질')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'지구과학과')
a = X[X['대학전공'].str.contains('천문|기상')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'천문학과')
a = X[X['대학전공'].str.contains('통계')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'통계학과')
a = X[X['대학전공'].str.contains('화학|화공|화확|Chemistry|회힉')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'화학과')

#### 공학계열

In [14]:
a = X[X['대학전공'].str.contains('정보통신')]['대학전공']
X['대학전공'] = X['대학전공'].replace(list(a),'정보통신공학과')
a = X[X['대학전공'].str.contains('컴퓨터|컴공|켬퓨터|컴정|서버|네트워크')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'컴퓨터공학과')
a = X[X['대학전공'].str.contains('소프트웨어|웹')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'소프트웨어공학과')
a = X[X['대학전공'].str.contains('신소재|신재생')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'신소재공학과')
a = X[X['대학전공'].str.contains('건축|설비')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'건축학과')
a = X[X['대학전공'].str.contains('게임')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'게임학과')
a = X[X['대학전공'].str.contains('기계')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'기계학과')
a = X[X['대학전공'].str.contains('메카트로')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'메카트로닉스학과')
a = X[X['대학전공'].str.contains('반도체|세라믹')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'반도체학과')
a = X[X['대학전공'].str.contains('산업공학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'산업공학과')
a = X[X['대학전공'].str.contains('섬유')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'섬유학과')
a = X[X['대학전공'].str.contains('광학|안경')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'안경학과')
a = X[X['대학전공'].str.contains('에너지')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'에너지학과')
a = X[X['대학전공'].str.contains('자동차')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'자동차학과')
a = X[X['대학전공'].str.contains('재료|금속')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'재료공학과')
a = X[X['대학전공'].str.contains('전기')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'전기학과')
a = X[X['대학전공'].str.contains('전자')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'전자학과')
a = X[X['대학전공'].str.contains('정보보안|보호')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'정보보안학과')
a = X[X['대학전공'].str.contains('제어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'제어학과')
a = X[X['대학전공'].str.contains('조경')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'조경학과')
a = X[X['대학전공'].str.contains('조경')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'조경학과')
a = X[X['대학전공'].str.contains('교통')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'교통공학과')
a = X[X['대학전공'].str.contains('토목')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'토목학과')
a = X[X['대학전공'].str.contains('해양')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'해양공학과')
a = X[X['대학전공'].str.contains('환경|환격')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'환경공학과')
a = X[X['대학전공'].str.contains('도시')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'도시공학과')

#### 예체능 계열

In [15]:
a = X[X['대학전공'].str.contains('경호')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'경호학과')
a = X[X['대학전공'].str.contains('공예')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'공예학과')
a = X[X['대학전공'].str.contains('만화|애니메이션')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'애니메이션학과')
a = X[X['대학전공'].str.contains('무용')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'무용학과')
a = X[X['대학전공'].str.contains('미술|한국화|회화')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'미술학과')
a = X[X['대학전공'].str.contains('방송연예')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'방송연예학과')
a = X[X['대학전공'].str.contains('사진|영상')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'사진영상학과')
a = X[X['대학전공'].str.contains('음악|관현악')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'음악학과')
a = X[X['대학전공'].str.contains('시각')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'시각학과')
a = X[X['대학전공'].str.contains('연극|영화')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'연극영화학과')
a = X[X['대학전공'].str.contains('체육|체욱|태권도')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'체육학과')
a = X[X['대학전공'].str.contains('조형')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'조형학과')
a = X[X['대학전공'].str.contains('패션|의류|의상')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'패션디자인학과')

#### 의학계열

In [16]:
a = X[X['대학전공'].str.contains('간호')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'간호학과')
a = X[X['대학전공'].str.contains('보건')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'보건학과')
a = X[X['대학전공'].str.contains('약학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'약학과')
a = X[X['대학전공'].str.contains('응급구조|구조')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'응급구조학과')
a = X[X['대학전공'].str.contains('의학|의과대학|의생활|의예|의학과')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'의학과')
a = X[X['대학전공'].str.contains('임상병리')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'임상병리학과')

#### 세부사항 추가변경

In [17]:
a = X[X['대학전공'].str.contains('수학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'수학과')
a = X[X['대학전공'].str.contains('이비즈니스|BUSINESS|비즈니스|e-|비지니스')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'비즈니스과')
a = X[X['대학전공'].str.contains('커뮤니케이션|Communication')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'커뮤니케이션')
a = X[X['대학전공'].str.contains('Management')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'관리학과')
a = X[X['대학전공'].str.contains('음악|Music|피아노')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'음악학과')
a = X[X['대학전공'].str.contains('가구')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'가구학과')
a = X[X['대학전공'].str.contains('건강')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'건강학과')
a = X[X['대학전공'].str.contains('건설공|건설시|건축|건충')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'건설학과')
a = X[X['대학전공'].str.contains('격기')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'격기학과')
a = X[X['대학전공'].str.contains('경기')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'경기지도학과')
a = X[X['대학전공'].str.contains('인류')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'고고인류학과')
a = X[X['대학전공'].str.contains('고분자')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'고분자학과')
a = X[X['대학전공'].str.contains('공간')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'공간학과')
a = X[X['대학전공'].str.contains('공업')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'공업학과')
a = X[X['대학전공'].str.contains('관경|관공통역|관좡')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'관광학과')
a = X[X['대학전공'].str.contains('통신공학|광통신')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'통신학과')
a = X[X['대학전공'].str.contains('교정학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'교육학과')
a = X[X['대학전공'].str.contains('국악')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'국악과')
a = X[X['대학전공'].str.contains('종합|선물보헙|선물')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'금융학과')
a = X[X['대학전공'].str.contains('기계|기게공')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'기계공학과')
a = X[X['대학전공'].str.contains('기공|기관공')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'기관공학과')
a = X[X['대학전공'].str.contains('기독교')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'기독교학과')
a = X[X['대학전공'].str.contains('기악')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'기악학과')
a = X[X['대학전공'].str.contains('축산|낙농')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'낙농학과')
a = X[X['대학전공'].str.contains('노문학|노어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'노어학과')
a = X[X['대학전공'].str.contains('신문방송학과|언론학|다중매체')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'매체학과')
a = X[X['대학전공'].str.contains('데이터')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'데이터학과')
a = X[X['대학전공'].str.contains('도서관|library')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'도서관학과')
a = X[X['대학전공'].str.contains('도예|도자')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'도예학과')
a = X[X['대학전공'].str.contains('독어독|독일')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'독일학과')
a = X[X['대학전공'].str.contains('동물자원|동물과학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'동물학과')
a = X[X['대학전공'].str.contains('동양무')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'동양무예학과')
a = X[X['대학전공'].str.contains('동양어문')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'동양학과')
a = X[X['대학전공'].str.contains('동양화')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'동양화')
a = X[X['대학전공'].str.contains('디지털')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'디지털학과')
a = X[X['대학전공'].str.contains('레저|레져')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'레저학과')
a = X[X['대학전공'].str.contains('마케팅')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'마케팅학과')
a = X[X['대학전공'].str.contains('목회')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'목회학과')
a = X[X['대학전공'].str.contains('무대디자인')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'무대디자인학과')
a = X[X['대학전공'].str.contains('미생물')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'미생물학과')
a = X[X['대학전공'].str.contains('법|law')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'법학과')
a = X[X['대학전공'].str.contains('보험')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'보험학과')
a = X[X['대학전공'].str.contains('부동산')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'부동산학과')
a = X[X['대학전공'].str.contains('분자생물')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'분자생물학과')
a = X[X['대학전공'].str.contains('불문|불어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'불어학과')
a = X[X['대학전공'].str.contains('사료')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'사료학과')
a = X[X['대학전공'].str.contains('사하|사회')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'사회학과')
a = X[X['대학전공'].str.contains('산업공|산업도예|산업시|산업안|산업재|산업정보')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'산업학과')
a = X[X['대학전공'].str.contains('산업 디자인|산업디자|산업제품디자인')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'산업디자인학과')  
a = X[X['대학전공'].str.contains('상경')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'상경학과')
a = X[X['대학전공'].str.contains('생명|생물|생물자원')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'생물학과')
a = X[X['대학전공'].str.contains('생산')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'생산학과')
a = X[X['대학전공'].str.contains('생활과|생활자원')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'생활학과')
a = X[X['대학전공'].str.contains('서양사|서양학과')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'서양학과')
a = X[X['대학전공'].str.contains('서양화')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'서양화')
a = X[X['대학전공'].str.contains('선교')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'선교학과')
a = X[X['대학전공'].str.contains('성악')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'성악학과')
a = X[X['대학전공'].str.contains('소비자 인간|소비자가족|소비자아동')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'소비자학과')
a = X[X['대학전공'].str.contains('소트웨어|소프트웨어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'소프트웨어학과')
a = X[X['대학전공'].str.contains('수족병리')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'수족병리학과')
a = X[X['대학전공'].str.contains('스포츠')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'스포츠학과')
a = X[X['대학전공'].str.contains('식량자원공학|식량자원학과')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'식량자원학과')
a = X[X['대학전공'].str.contains('신학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'신학과')
a = X[X['대학전공'].str.contains('아동|유아')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'아동학과')
a = X[X['대학전공'].str.contains('안전')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'안전학과')
a = X[X['대학전공'].str.contains('애니')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'애니학과')
a = X[X['대학전공'].str.contains('양식')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'양식학과')
a = X[X['대학전공'].str.contains('연극영화|연기|Theatre')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'연기학과')
a = X[X['대학전공'].str.contains('예술대학|예술학부|체능')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'예술학과')
a = X[X['대학전공'].str.contains('요업공학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'요업공학과')
a = X[X['대학전공'].str.contains('운동기능|웨이트트레이닝|헬스')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'운동학과')
a = X[X['대학전공'].str.contains('유도')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'유도학과')
a = X[X['대학전공'].str.contains('유전공')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'유전공학과')
a = X[X['대학전공'].str.contains('유통')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'유통학과')
a = X[X['대학전공'].str.contains('유학과|유학동양')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'유학')
a = X[X['대학전공'].str.contains('소재')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'소재학과')
a = X[X['대학전공'].str.contains('인터넷')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'인터넷학과')
a = X[X['대학전공'].str.contains('인테리어')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'인테리어학과')
a = X[X['대학전공'].str.contains('일본|일문')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'일본학과')
a = X[X['대학전공'].str.contains('임상|임학')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'임상학과')
a = X[X['대학전공'].str.contains('자원')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'자원학과')
a = X[X['대학전공'].str.contains('작곡')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'작곡학과')
a = X[X['대학전공'].str.contains('전산')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'전산학과')
a = X[X['대학전공'].str.contains('전파|전자')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'전파학과')
a = X[X['대학전공'].str.contains('정보')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'정보학과')
a = X[X['대학전공'].str.contains('제약')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'제약학과')
a = X[X['대학전공'].str.contains('조소')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'조소학과')
a = X[X['대학전공'].str.contains('주거솬|주생활')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'주거학과')
a = X[X['대학전공'].str.contains('지식정보')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'지식정보학과')
a = X[X['대학전공'].str.contains('지역개발|지역계획|지역정보')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'지역학과')
a = X[X['대학전공'].str.contains('청소년')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'청소년학과')
a = X[X['대학전공'].str.contains('양식')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'축산학과')
a = X[X['대학전공'].str.contains('토&#4|토목학|토옥')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'토목공과')
a = X[X['대학전공'].str.contains('수원대|&#49887;품공학|1|2급|3.|F-H|Summer|Material|SOC|clothing|ddd|ghj|tesol|ㅇ|없음|이공계|졸업|준사서|증식|철철|해선|eh|경매|더블|상학과|마이컴|Social|광보')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'기타')
a = X[X['대학전공'].str.contains('복식디자인|비주얼|생활디자인|서피스|스페이스|실내디자인|실내장식|써피스|fashoin|일러스트|장신구|제품디자인|텍스타일|패션|편집|지자인')]['대학전공'] 
X['대학전공'] = X['대학전공'].replace(list(a),'디자인학과')

In [18]:
# 이상값 '기타'로 처리
etc_lst = ['철', '새왈과학부', '외식사업', '설계전공', '영', '산', '사체']

for val in etc_lst:
    idx = X['대학전공'][X['대학전공'] == val].index
    X['대학전공'].loc[idx] = '기타'
    

#### 어학시험 동일 시험들 하나의 시험명으로 통일

In [19]:
X = X.replace({'어학시험': {'TOEFL-CBT','TOEFL-IBT','TOEFL-PBT','TORFL'}}, 'TOEFL')
X = X.replace({'어학시험': '新HSK'}, 'HSK')
X = X.replace({'어학시험': {'JPT','JTRA'}}, 'JLPT')
X = X.replace({'어학시험': 'TOEIC S&W'}, 'TOEIC Speaking')
X = X.replace({'어학시험': 'Opi'}, 'OPIc')
X = X.replace({'어학시험': 'G-TELP(GLT)'}, 'G-TELP')
X = X.replace({'어학시험': 'DELF'}, 'DALF')

In [20]:
# X.to_csv('./pre_processing.csv', encoding='cp949', index=False)